In [1]:
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai
import json

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from flaml.autogen.code_utils import extract_code
from create_bench_agent import CreateBenchAgent

In [2]:
example_qa = """
----------
Param Update: ```JSON
"end_charge_time": 9
```
Example User request: Charge the car till 9 AM.

----------
Param Update: ```JSON
"end_charge": 1.00,
"end_charge_time": 9
```
Example Request: Charge the car to full charge by 9 AM.
"""

In [3]:
params_filepath = "/Users/viditjain/Desktop/LLM Optimizer/EnergySaverLLM/EnergySaverLLM/Model/params/EVCharging.json"

In [4]:
log_history = {}

In [5]:
autogen.oai.ChatCompletion.start_logging(log_history)

In [6]:
config_list = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": [ "gpt-4","gpt-3.5-turbo-16k"],
    },
)

In [7]:
agent = CreateBenchAgent(
    name="CreateBenchAgent",
    example_qa=example_qa,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

In [8]:
with open('benchmark_values.json', 'r') as f:
    # text = f.read()
    values_db = json.loads(f.read())

In [9]:
param_update = """```JSON
{key}: {new_val}
```"""

In [10]:
benchmark_dataset = []
for key, val_list in values_db.items():
    for new_val in val_list:
        benchmark_instance = {}
        benchmark_instance["json_str"] = param_update.format(key = key, new_val = new_val)
        agent = CreateBenchAgent(
            name="CreateBenchAgent",
            example_qa=example_qa,
            json_filepath=params_filepath,
            llm_config={
                "request_timeout": 600,
                "seed": 42,
                "config_list": config_list,
            }
        )
        user = UserProxyAgent(
            "user", max_consecutive_auto_reply=0,
            human_input_mode="NEVER", code_execution_config=False
        )
        
        user.initiate_chat(agent, message = benchmark_instance["json_str"], silent = True)

        log_list = []
        log_list = eval(list(log_history.keys())[-1])
        benchmark_instance['prompt'] = ""+(log_list[-1]['content'])

        benchmark_dataset.append(benchmark_instance)

carbon_cost_weight
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Lower the factor for the carbon cost's influence on the total electricity cost.

--------------------------------------------------------------------------------
carbon_cost_weight
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

I would like to increase the weight for carbon cost in total cost calculation to 0.4.

--------------------------------------------------------------------------------
max_energy_cost
CreateBenchAgent (to writer):




Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Decrease the maximum electricity cost allowed in one charge cycle to 10 USD.

----------------------------

In [29]:
benchmark_dataset

[{'json_str': '```JSON\ncarbon_cost_weight: 0.2\n```',
  'prompt': 'Increase the weightage of carbon cost in total cost to 0.4.'},
 {'json_str': '```JSON\ncarbon_cost_weight: 0.4\n```',
  'prompt': 'Increase the weightage of carbon cost in total cost to 0.4.'},
 {'json_str': '```JSON\nmax_energy_cost: 10\n```',
  'prompt': 'Lower the maximum electricity cost allowed in one charge cycle to $10.'},
 {'json_str': '```JSON\nmax_energy_cost: 15\n```',
  'prompt': 'Lower the max energy cost to $15 during one charging cycle.'},
 {'json_str': '```JSON\nmax_energy_cost: 20\n```',
  'prompt': 'Increase the maximum energy cost allowed for one charging cycle to $20.'},
 {'json_str': '```JSON\nmax_energy_cost: 25\n```',
  'prompt': 'Increase the maximum energy cost allowed in one charge cycle to $25.'},
 {'json_str': '```JSON\nmax_energy_cost: 30\n```',
  'prompt': 'Increase the maximum electricity cost allowed in one charge cycle to 30 USD.'},
 {'json_str': '```JSON\nmax_energy_cost: 35\n```',
  '

In [30]:
benchmark_dataset_filename = "EV.benchmark.json"

In [33]:
with open(benchmark_dataset_filename, "w") as text_file:
    text_file.write(json.dumps(benchmark_dataset, indent=2))

In [6]:
config_list = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": [ "gpt-3.5-turbo-16k"],
    },
)

In [7]:
param_update = """
```JSON
"carbon_cost_weight": 0.5
```
"""

In [8]:
agent = CreateBenchAgent(
    name="CreateBenchAgent",
    example_qa=example_qa,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

In [9]:
user.initiate_chat(agent, message = param_update, silent=True)

CreateBenchAgent (to writer):


The JSON for updated parameter
{param_update}

Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Change the `carbon_cost_weight` parameter to `0.5`.

--------------------------------------------------------------------------------


In [12]:
user.initiate_chat(agent, message = param_update, silent=True)

CreateBenchAgent (to writer):


The JSON for updated parameter
{param_update}

Describe the change:



--------------------------------------------------------------------------------
writer (to CreateBenchAgent):

Adjust the weight of carbon emissions to 0.5 in the total charging cost.

--------------------------------------------------------------------------------


'I want to change the weight given to carbon cost to 0.5.'